In [98]:
from transformers import DecisionTransformerModel, DecisionTransformerConfig
import numpy as np
import torch

device = "cuda"

scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 3600 / scale  # evaluation conditioning targets, 3600 is reasonable from the paper LINK
state_dim = (3, 32, 32)
state_dim_flatten = state_dim[0]*state_dim[1]*state_dim[2]
act_dim = 6

In [100]:
n_positions=1024*4
max_ep_len=1024 # no impact?

t = int(n_positions/3) # number of elements in sequence
b = 1 # batch 

target_return = torch.rand((b, t, 1), device=device, dtype=torch.float32) # expected future return
states = torch.rand((b, t, state_dim_flatten), device=device, dtype=torch.float32)#.reshape(1, 1, state_dim_flatten) #
actions = torch.rand((b, t, act_dim), device=device, dtype=torch.float32)

rewards = torch.rand((b,2, t), device=device, dtype=torch.float32) # doesnt matter idk why exists
timesteps = torch.randint_like(torch.zeros((b, t)), device=device, low = 100, high = max_ep_len, dtype=torch.long) # integer what timestep we on


attention_mask = torch.ones((b, t), device=device, dtype=torch.long) # this is the default

print("target_return",target_return.shape)
print("states",states.shape)
print("actions",actions.shape)
print("rewards",rewards.shape)
print("timesteps", timesteps.shape)
print("attention_mask", attention_mask.shape)

config = DecisionTransformerConfig(state_dim_flatten, act_dim, max_ep_len=max_ep_len,n_positions=n_positions)


model = DecisionTransformerModel(config).to(device)
model.eval()
with torch.no_grad():
        state_preds, action_preds, return_preds = model(states=states,
                actions=actions,
                rewards=rewards,
                returns_to_go=target_return,
                timesteps=timesteps,
                attention_mask=attention_mask,
                return_dict=False)

        #print("argmax", torch.argmax(action_preds).item())
        print("state_preds",state_preds.shape)
        print("action_preds",action_preds.shape)
        print("return_preds", return_preds.shape)


        print("state_preds", state_preds[0])
        print("action_preds", action_preds[0])
        print("return_preds", return_preds[0])

target_return torch.Size([1, 1365, 1])
states torch.Size([1, 1365, 3072])
actions torch.Size([1, 1365, 6])
rewards torch.Size([1, 2, 1365])
timesteps torch.Size([1, 1365])
attention_mask torch.Size([1, 1365])
state_preds torch.Size([1, 1365, 3072])
action_preds torch.Size([1, 1365, 6])
return_preds torch.Size([1, 1365, 1])
state_preds tensor([[ 0.0590,  0.1822, -0.0669,  ...,  0.1140,  0.4109, -0.2156],
        [ 0.1704,  0.1097, -0.2572,  ...,  0.1839, -0.1295, -0.1316],
        [ 0.1030,  0.1238,  0.0213,  ...,  0.0535,  0.2225,  0.0342],
        ...,
        [ 0.0052,  0.3805, -0.0504,  ..., -0.0646,  0.4899, -0.3576],
        [ 0.2908,  0.2081, -0.1710,  ...,  0.1995, -0.0806,  0.0363],
        [ 0.0445,  0.2653, -0.0387,  ...,  0.0254,  0.4619, -0.2510]],
       device='cuda:0')
action_preds tensor([[-0.0676,  0.1410,  0.0733, -0.1185,  0.0715,  0.2120],
        [ 0.0154, -0.0313,  0.1795, -0.2725, -0.1355,  0.4287],
        [ 0.1194,  0.0815,  0.2098, -0.4338, -0.1554,  0.3901],


In [ ]:
import random

population = [0, 1, 2, 3, 4]
ls = random.sample(population,5)

print("Pure random: ", ls)

max_sequence = 4

def sample_with_order(population, max_sequence_length):
    """
        Samples [1,population] elements from a sequence mantaining the order
    """
    k = random.randint(1, max_sequence_length)
    k_start = random.randint(0, len(population)-k)
    return population[k_start: k_start+k]


print(sample_with_order(population, max_sequence))

   

Pure random:  [0, 1, 4, 2, 3]
[1, 2, 3, 4]


In [ ]:
target_return = torch.ones(1, device=device, dtype=torch.float32).reshape(100, 1)

RuntimeError: shape '[100, 1]' is invalid for input of size 1